# Non-adaptive Scalar [Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)) of Digital Signals

In [ ]:
%matplotlib inline

import math
import numpy as np

import common
!ln -sf ~/repos/image_IO/logging_config.py .

## A.  Uniform Quantization

### A.a. [Mid-tread ("round") Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Example)

Mid-tread quantizers are uniform quantizers (the decision levels are spaced evenly) and has zero as one of its output (dequantized) levels.

Quantization index:
\begin{equation}
k = \left\lfloor \frac{\left| x \right|}{\Delta} + \frac{1}{2}\right\rfloor.
\end{equation}

Reconstructed value:
\begin{equation}
  y = \Delta k.
\end{equation}

The $k$ index can be also computed using the [round half toward zero](https://en.wikipedia.org/wiki/Rounding#Round_half_towards_zero) (or round to the nearest integer), for which NumPy provides the method [rint()](https://numpy.org/doc/stable/reference/generated/numpy.rint.html).

In [ ]:
!cat midtread_quantization.py
import midtread_quantization as midtread

In [ ]:
Q_step = 2
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
x = np.linspace(-8, 8, 500) # Input samples
y, k = Q.quan_dequan(x)
common.plot(x, y, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))

The reconstruction levels are in the middle of each bin (this is true for all the non-adaptive quantizers). For example, the values between -1 and 1 ($\Delta=2$) are reconstructed as 0. The values between 1 and 3, are reconstructed by 2.

Depending on the resolution of the input (digital) signal $x$ and quantization step size $\Delta$, the bins can have different size:

In [ ]:
x = np.arange(15)-7
y, k = Q.quan_dequan(x)
print(x)
print(k)
print(y)

... but this effect is only noticeable if the resolution of $x$ and the size of $\Delta$ are comparable.

### A.b. [Mid-tread Quantization with Deadzone](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Dead-zone_quantizers)

Deadzone quantizers are characterized by using a bin of size $2\Delta$ centered at the 0 input. See *JPEG2000 Image Compression Fundamentals, Standards and Practice*.

Quantization index
\begin{equation}
k = \left\{ 
  \begin{array}{cl}
    \text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta}\right\rfloor & \quad \text{if } \frac{\left| x \right|}{\Delta} >0 \\
    0                 & \quad \textrm{otherwise},
  \end{array}
\right.
\end{equation}
which can be computed efficiently in NumPy by simply converting the floating point representation of $x/\Delta$ to an integer using the [astype()](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) method.
Reconstructed value

\begin{equation}
y = \left\{ 
  \begin{array}{cl}
    0                                          & \quad \text{if } k=0 \\
    \text{sign}(k) (\left| k \right|-0.5)\Delta & \quad \text{if } k\ne 0 
  \end{array}
\right.
\end{equation}

Using the same simplification, the second equation boils down to
\begin{equation}
  y = \Delta k.
\end{equation}


In [ ]:
!cat deadzone_quantization.py
import deadzone_quantization as deadzone

In [ ]:
Q_step = 2
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
x = np.linspace(-8, 8, 500) # Input samples
y, k = Q.quan_dequan(x)
common.plot(x, y, "Input Sample", "Reconstructed Sample", "Mid-tread (with Deadzone) Quantizer ($\Delta={}$)".format(Q_step))

In [ ]:
x = np.arange(15)-7
y, k = Q.quan_dequan(x)
print(x)
print(k)
print(y)

### A.c. [Mid-rise ("truncation") Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Mid-riser_and_mid-tread_uniform_quantizers)

Mid-rise quantizers do not have zero as one of its representation levels (in fact, at 0, the output is $\Delta/2$).

Quantization index
\begin{equation}
  k=\Big\lfloor \frac{x}{\Delta}\Big\rfloor.
\end{equation}

Reconstructed value
\begin{equation}
  y = \Delta \Big(k + \frac{1}{2}\Big).
\end{equation}

In [ ]:
!cat midrise_quantization.py
import midrise_quantization as midrise

In [ ]:
Q_step = 2
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
x = np.linspace(-8, 8, 500) # Input samples
y, k = Q.quan_dequan(x)
common.plot(x, y, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))

In [ ]:
x = np.arange(15)-7
y, k = Q.quan_dequan(x)
print(x)
print(k)
print(y)

## B. [Non Uniform Quantization](https://nptel.ac.in/content/storage2/courses/117104069/chapter_5/5_5.html)

### B.a. [Companded (COMpressed + exPANDED)](https://en.wikipedia.org/wiki/Companding) Quantization

The bins are smaller at the input 0 and viceversa. This can be interesing if most of the samples are small (in absolute value), because we will minimize the quantization error.

#### B.a.1. [$\mu$-Law](https://en.wikipedia.org/wiki/%CE%9C-law_algorithm) Companded  Quantization

Compressor
\begin{equation}
C(x) = \text{sgn}(x) \frac{\ln(1+ \mu |x|)}{\ln(1+\mu)}, ~~~~-1 \leq x \leq 1,
\end{equation}
shere $\mu=255$ in most implementations.

Expander:
\begin{equation}
C^{-1}(y) = \text{sgn}(y) (1 / \mu ) ((1 + \mu)^{|y|}- 1),~~~~-1 \leq y \leq 1.
\end{equation}

In [ ]:
!cat companded_quantization.py
import companded_quantization as companded

In [ ]:
x = np.linspace(-1, 1, 500)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)

mu = 255
y = Q.muLaw_compress(x, mu)
common.plot(x, y, "Input", "Output", "$\mu$-law Compressor ($\mu={}$)".format(mu))

In [ ]:
x = np.linspace(-1, 1, 500)

mu = 255
y = Q.muLaw_expand(x, mu)
common.plot(x, y, "Input", "Output", "$\mu$-law Expander ($\mu={}$)".format(mu))

In [ ]:
mu = 255
x = np.linspace(-1, 1, 500)
y = Q.muLaw_compress(x, mu)
x_recons = Q.muLaw_expand(y, mu)
common.plot(x, x_recons, "Input", "Output", "Expansion(Compression(Input))".format(mu))

After these definitions, we define the quantization index
\begin{equation}
  k = Q\big(C(x)\big),
\end{equation}
where $C$ is the compression function and $Q$ is a dead-zone quantizer. 

Reconstruction value
\begin{equation}
  y = C^{-1}\big(Q^{-1}(k)\big),
\end{equation}
where $Q^{-1}$ stands for the dead-zone dequantizer and $C^{-1}$ for the expander function.

In [ ]:
Q_step = 1024
min_val = -32767
max_val = 32767
x = np.linspace(min_val, max_val, 500)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=min_val, max_val=max_val)
y, k = Q.quan_dequan(x)
common.plot(x, y, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
# Ignore the rest.

import time
while True:
    time.sleep(1)

### A.c. [Dead-zone Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Dead-zone_quantizers)

See *JPEG2000 Image Compression Fundamentals, Standards and Practice*.

Quantization index
\begin{equation}
k = \left\{ 
  \begin{array}{cl}
    \text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta}\right\rfloor & \quad \text{if } \frac{\left| x \right|}{\Delta} >0 \\
    0                 & \quad \textrm{otherwise},
  \end{array}
\right.
\end{equation}
which can be computed efficiently in NumPy by simply converting the floating point representation of $x/\Delta$ to an integer using the [astype()](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) method.
Reconstructed value

\begin{equation}
y = \left\{ 
  \begin{array}{cl}
    0                                          & \quad \text{if } k=0 \\
    \text{sign}(k) (\left| k \right|-0.5)\Delta & \quad \text{if } k\ne 0 
  \end{array}
\right.
\end{equation}

Using the same simplification, the second equation boils down to
\begin{equation}
  y = \Delta k.
\end{equation}


In [ ]:
!cat deadzone_quantization.py
import deadzone_quantization as deadzone

In [ ]:
x = np.arange(11)-5

In [ ]:
print("x =", x)
print("k =", np.rint(x/Q_step).astype(np.int16))

In [ ]:
Q.quantize(x)

In [ ]:
x = np.array([65])
Q_step = 128
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
k = Q.quantize(x)
y = Q.dequantize(k)
print("k =", k)
print("y =", y)

In [ ]:
print("x =", x)
print("k =", np.floor(x/Q_step).astype(np.int16))

In [ ]:
x = np.array([65])
Q_step = 128
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
k = Q.quantize(x)
y = Q.dequantize(k)
print("k =", k)
print("y =", y)

### A.c. [Dead-zone Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Dead-zone_quantizers)

See *JPEG2000 Image Compression Fundamentals, Standards and Practice*.

Quantization index
\begin{equation}
k = \left\{ 
  \begin{array}{cl}
    \text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta}\right\rfloor & \quad \text{if } \frac{\left| x \right|}{\Delta} >0 \\
    0                 & \quad \textrm{otherwise},
  \end{array}
\right.
\end{equation}
which can be computed efficiently in NumPy by simply converting the floating point representation of $x/\Delta$ to an integer using the [astype()](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) method.
Reconstructed value

\begin{equation}
y = \left\{ 
  \begin{array}{cl}
    0                                          & \quad \text{if } k=0 \\
    \text{sign}(k) (\left| k \right|-0.5)\Delta & \quad \text{if } k\ne 0 
  \end{array}
\right.
\end{equation}

Using the same simplification, the second equation boils down to
\begin{equation}
  y = \Delta k.
\end{equation}


In [ ]:
!cat deadzone_quantization.py
import deadzone_quantization as deadzone

In [ ]:
print("x =", x)
print("k =", np.sign(x)*np.floor(np.abs(x)/Q_step).astype(np.int16))
print("k =", (x/Q_step).astype(np.int16))

In [ ]:
x = np.array([127])
Q_step = 128
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=-128, max_val=127)
k = Q.quantize(x)
y = Q.dequantize(k)
print("k =", k)
print("y =", y)

## Comparing Quantizers I/O

In [ ]:
Q_step = 1 # Delta
x = np.linspace(-8, 8, 500) # Input samples
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
Q_step = 2
x = np.linspace(-8, 8, 500)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
Q_step = 3
x = np.linspace(-8, 8, 500)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
Q_step = 4
x = np.linspace(-8, 8, 500)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
Q_step = 1024
min_val = -32768
max_val = 32767
x = np.linspace(min_val, max_val, 500)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=min_val, max_val=max_val)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=min_val, max_val=max_val)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=min_val, max_val=max_val)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=min_val, max_val=max_val)
y_C, k_C = Q.quan_dequan(x)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(Q_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, Q_step))

## Working with signed integers of 16 bits

In [ ]:
Q_step = 1
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")


In [ ]:
Q_step = 2
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")


In [ ]:
Q_step = 32
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
Q = midtread.Midtread_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_T, k_T = Q.quan_dequan(x)
Q = midrise.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_R, k_R = Q.quan_dequan(x)
Q = deadzone.Deadzone_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_D, k_D = Q.quan_dequan(x)
Q = companded.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
y_C, k_C = Q.quan_dequan(x)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")
